## **Escenario 3: Equipo de ML Distribuido con Infraestructura en la Nube**

### **Contexto del Escenario**
Este notebook simula un entorno de producción donde múltiples científicos de datos trabajan en múltiples modelos de ML desde diferentes ubicaciones, utilizando una infraestructura completamente distribuida en la nube.

### **Configuración de MLflow Enterprise**
- **Tracking server**: Servidor remoto en EC2 (AWS)
- **Backend store**: Base de datos PostgreSQL
- **Artifacts store**: Bucket S3 de AWS
- **Model Registry**: Completamente funcional para gestión de modelos


## **Requisitos Previos**

### **Infraestructura AWS**
Para ejecutar este ejemplo necesitarás:
- Una cuenta de AWS activa
- Configuración de credenciales AWS
- Instancia EC2 ejecutando el servidor MLflow
- Base de datos PostgreSQL configurada
- Bucket S3 para almacenamiento de artifacts

## **Configuración de cliente**

In [ ]:
import mlflow
import os

os.environ["AWS_PROFILE"] = "" # fill in with your AWS profile. More info: https://docs.aws.amazon.com/sdk-for-java/latest/developer-guide/setup.html#setup-credentials

TRACKING_SERVER_HOST = "" # fill in with the public DNS of the EC2 instance
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [ ]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

In [ ]:
mlflow.search_experiments() # list_experiments API has been removed, you can use search_experiments instead.()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

In [ ]:
mlflow.search_experiments()

Interacción con el Model Registry

In [ ]:
from mlflow.tracking import MlflowClient


client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [ ]:
client.search_registered_models()

In [ ]:
run_id = client.search_runs(experiment_ids=['1'])[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

## **Diferencias con Escenarios Anteriores**

### **Escenario 1 (Local)**
- Archivos locales, sin servidor
- Sin colaboración, uso individual

### **Escenario 2 (Servidor Local)**
- Servidor local con SQLite
- Colaboración limitada a red local

### **Escenario 3 (Nube)**
- Servidor remoto en EC2
- Colaboración global, acceso desde cualquier lugar
- Infraestructura escalable y robusta

---

## **Ventajas del Escenario 3**

1. **Colaboración Global**: Equipos distribuidos pueden trabajar desde cualquier ubicación
2. **Escalabilidad**: Infraestructura que crece con las necesidades del equipo
3. **Confiabilidad**: Base de datos PostgreSQL robusta y almacenamiento S3 duradero
4. **Seguridad**: Control de acceso a nivel de AWS IAM
5. **Integración**: Fácil integración con pipelines de CI/CD y herramientas de DevOps
6. **Monitoreo**: Logs y métricas de infraestructura disponibles en CloudWatch